# Задача:
Взять топ 150 стран по объёму импорта и зеркального импорта товара в 2019 году, Россию туда добавить в любом случае. 

Для каждого месяца с 2015 по 2023 год должна быть информация - страна импортёр, страна из которой был ввезён импорт, объёмы физические и стоимостные для каждой комбинации

# Импорт модулей

In [1]:
import time
import os
import pandas as pd
import requests

# Коды UN-49M

Это коды соответствия названия страны/обьединения и их кода

In [2]:
URL = 'https://comtradeapi.un.org/files/v1/app/reference/Reporters.json'

response = requests.get(URL)
data = response.json()

m49codeToName = dict()

for country in data['results']:
    reporterCode = country['id']
    name = country['text']
    isGroup = country['isGroup']

    if not isGroup:
        m49codeToName[reporterCode] = name

для примера, код Российской федерации - 643

In [3]:
m49codeToName[643]

'Russian Federation'

# Топ 150 стран по обьему импорта бензина в 2019 году

Получим список стран для которых будем собирать статистику. 2019 год взят за основу

Определимся с **hs_code** - кодом товара, по которому будем собирать статистику и зеркальную статистику

In [4]:
# 271019: нефтепродукты: средние и тяжелые дистилляты – дизельное топливо, керосин, мазут
hs_code = 271019

In [65]:
# эти ключи регулярно обновляются и удаляются, поэтому не страшно их выкладывать
PRIMARY_KEY = '9f1492872e0c48e4849fe94ab8cd1332' 

In [66]:
typeCode = 'C' # Товары
freqCode = 'A' # Данные по годам
clCode = 'HS'  

URL = f'https://comtradeapi.un.org/data/v1/get/{typeCode}/{freqCode}/{clCode}' 

params = {
    'subscription-key': PRIMARY_KEY, # Ключ доступа к API Comtrade
    'partnerCode':      0,           # Код UN M49 для всего мира
    'period':           2019,
    'flowCode':         'M',         # Импорт
    'cmdCode':          hs_code      #
}

In [7]:
response = requests.get(URL, params)

response_data = response.json()['data']
df = pd.json_normalize(response_data)

df['Country'] = df.reporterCode.apply(lambda key: m49codeToName.get(key, ''))

In [8]:
top150 = df.sort_values(by='primaryValue', ascending=False)\
    .drop_duplicates(subset=['reporterCode'])\
    [['Country', 'reporterCode', 'primaryValue']]\
    .reset_index(drop=True)\
    .head(150)

In [9]:
top150.head(10)

,Country,reporterCode,primaryValue
0,USA,842,3.631031e+10
1,Singapore,702,3.196973e+10
2,France,251,2.028984e+10
3,Germany,276,1.743834e+10
4,United Kingdom,826,1.648988e+10
5,Netherlands,528,1.628514e+10
6,Australia,36,1.467114e+10
7,China,156,1.279018e+10
8,Mexico,484,1.219187e+10
9,"China, Hong Kong SAR",344,1.085113e+10


### Проверяем что там есть РФ

In [10]:
top150[top150.reporterCode == 643]

,Country,reporterCode,primaryValue
77,Russian Federation,643,8.342934e+08


# Создаем список стран

In [11]:
CountryID = list(top150.reporterCode)

In [12]:
type(CountryID[0])

int

# Соберем статистику импорта

Для каждого месяца с 2015 по 2023 год должна быть информация - страна Экспортер, страна в которую был ввезён экспорт, объёмы физические и стоимостные для каждой комбинации

По скольку Comtrade API блокирует запросы которые поступают чаще, чем раз в 10 секунд, то все данные собираются синхронно

*Для асинхронной работы можно получить специальный доступ у них на сайте*

In [13]:
min_year = 2015
max_year = 2023

years = range(min_year, max_year + 1)
months = [f"{month:02}" for month in range(1, 12 + 1)]

### сформируем запрос

он будет немного отличаться, чтобы не забывать его структуру, дублирую

In [67]:
typeCode = 'C' # Товары
freqCode = 'M' # Данные по месяцам
clCode = 'HS'  # Тип товара 

URL = f'https://comtradeapi.un.org/data/v1/get/{typeCode}/{freqCode}/{clCode}' 

params = {
    'subscription-key': PRIMARY_KEY, # Subscription key 
    'flowCode':         'X',         # Export
    'cmdCode':          hs_code,     #
    'reporterCode':     ','.join(map(str, CountryID))
}

In [68]:
def pull_data(params):
    """получение pd.DataFrame по запросу Comtrade API"""
    
    response = requests.get(URL, params)

    try:
        response_data = response.json()['data']
        df = pd.json_normalize(response_data)
    except Exception as exc:
        raise Exception(f'cant normalize data: {exc}, response: {response.json()}')
    return df        

# Сформируем модель получаемых данных

Данные будут сохраняться в датафрейм со следующими столбцами:
- period: временной промежуток времени данной статистики
- reporterCode: код страны, из которой проводился экспорт
- primaryValue: стоимость поставки (долл. США)
- netWgt: физический обьем

In [69]:
data = pd.DataFrame({'period': [], 'reporterCode': [], 'primaryValue': [], 'netWgt': []})

# Парсинг данных

In [70]:
for i, year in enumerate(years):
    period = ','.join(str(year) + str(month) for month in months)
    params['period'] = period
    response = requests.get(URL, params)    
    
    try:
        response_data = response.json()['data']
        df = pd.json_normalize(response_data)

    except Exception as exc:
        print(f'\t\tPULLING STOPPED\n\nPulling stopped with exception: {exc}\n\nData saved to ./ABORTED DATA: {hash(period)}.pkl')
        data.to_pickle(f'ABORTED DATA: {hash(period)}.pkl')
        break
    
    periods = df['period'].unique()
    
    for period_ in periods:
        for reporterCode in CountryID:
            filtered_df = df[(df['reporterCode'] == reporterCode) & (df['period'] == period_)]
            
            partners = list(filtered_df['partnerCode'])
            wgts = list(filtered_df['netWgt'])
            values = list(filtered_df['primaryValue'])
    
            primaryValues = {country: value for country, value in zip(partners, values)}
            netWgts = {country: weight for country, weight in zip(partners, wgts)}
    
            row = pd.DataFrame({'period': [period_], 'reporterCode': [reporterCode], 'primaryValue': [primaryValues], 'netWgt': [netWgts]})
            data = pd.concat([data, row])

    print(f'{(i + 1) /  9 * 100:0.2f}%\t{year = }')                           

11.11%	year = 2015
22.22%	year = 2016
33.33%	year = 2017
44.44%	year = 2018
55.56%	year = 2019
66.67%	year = 2020
77.78%	year = 2021
88.89%	year = 2022
100.00%	year = 2023


In [72]:
data.period.unique()

array(['201501', '201502', '201503', '201504', '201505', '201506',
       '201507', '201508', '201509', '201510', '201511', '201601',
       '201602', '201603', '201604', '201605', '201606', '201607',
       '201608', '201609', '201610', '201701', '201702', '201703',
       '201704', '201705', '201706', '201707', '201708', '201801',
       '201802', '201803', '201804', '201805', '201806', '201807',
       '201901', '201902', '201903', '201904', '201905', '201906',
       '201907', '202001', '202002', '202003', '202004', '202005',
       '202006', '202007', '202008', '202101', '202102', '202103',
       '202104', '202105', '202106', '202107', '202108', '202201',
       '202202', '202203', '202204', '202205', '202206', '202207',
       '202301', '202302', '202303', '202304', '202305', '202306',
       '202307'], dtype=object)

In [26]:
data.dropna(subset=['reporterCode'], inplace=True)

In [27]:
data.to_csv(f'graph data imp - {hs_code}.csv')

In [19]:
data = pd.read_csv(f'graph data imp - {hs_code}.csv', index_col='Unnamed: 0')

In [20]:
data

,period,reporterCode,primaryValue,netWgt
0,201501,842.0,"{124: 413074932.0, 484: 562456657.0, 826: 1478...","{124: 0.0, 484: 0.0, 826: 0.0, 156: 0.0, 392: ..."
0,201501,702.0,"{430: 9.72, 0: 2432203650.161, 360: 141837375....","{430: 0.0, 0: 0.0, 360: 240826090.0, 31: 17430..."
0,201501,251.0,"{705: 18168329.505, 703: 189945.245, 100: 1011...","{705: 33090308.0, 703: 82427.0, 100: 63437.0, ..."
0,201501,276.0,"{842: 1277.148, 699: 363.737, 490: 1622493.805...","{842: 716.0, 699: 13.0, 490: 527969.0, 579: 70..."
0,201501,826.0,"{0: 380702328.349, 40: 64374.634, 348: 65099.6...","{0: 890924514.0, 40: 20444.0, 348: 17766.0, 30..."
...,...,...,...,...
0,202307,384.0,{},{}
0,202307,662.0,{},{}
0,202307,308.0,"{0: 1640.37, 780: 1640.37}","{0: 50.0, 780: 50.0}"
0,202307,140.0,{},{}


# Зеркальная статистика

In [21]:
typeCode = 'C' # Товары
freqCode = 'M' # Данные по месяцам
clCode = 'HS'  #

URL = f'https://comtradeapi.un.org/data/v1/get/{typeCode}/{freqCode}/{clCode}' 

params = {
    'subscription-key': PRIMARY_KEY, # Subscription key
    'flowCode':         'X',         # Export
    'cmdCode':          hs_code,     #
    'partnerCode':      None,
    'reporterCode':     ','.join(map(str, CountryID))
}

In [22]:
def pull_data(params):    
    response = requests.get(URL, params)

    response_data = response.json()['data']
    
    df = pd.json_normalize(response_data)
    
    return df        

## Инициализируем датафрейм

Заполним периоды и коды репортеров, т.к. эти данные статические

зеркальную статистсику оставим пустыми значениями,так понятней сколько собрали и будет проще заполнять пропуски 

In [23]:
mirror_data = pd.DataFrame({
        'period': [f'{year}{month}' for year in years for _ in CountryID for month in months],
        'reporterCode': [ceporterCode for year in years for ceporterCode in CountryID for month in months],
        'mirrorPrimaryValue': [None for year in years for ceporterCode in CountryID for month in months],
        'mirrorNetWgt': [None for year in years for ceporterCode in CountryID for month in months]
    })

In [54]:
# Если уже сохраняли результат
# mirror_data = pd.read_csv(f'graph data mirrored imp - {hs_code}.csv', index_col='Unnamed: 0')
# mirror_data.period = mirror_data.period.astype('str')

In [28]:
mirror_data.head(3)

,period,reporterCode,mirrorPrimaryValue,mirrorNetWgt
0,201501,842,"{32: 20.0, 36: 14213.961, 40: 6345.066, 56: 47...","{32: 31.2, 36: 0.0, 40: 0.0, 56: 121257385.254..."
1,201502,842,"{352: 144687.791, 360: 2780.0, 372: 10427.245,...","{352: 203952.0, 360: 2813.176, 372: 5407.426, ..."
2,201503,842,"{604: 9238106.87, 579: 22806402.843, 191: 3798...","{604: 13515588.0, 579: 54169094.0, 191: 1002.0..."


In [30]:
min_year = 2015
max_year = 2023

years = range(min_year, max_year + 1)

months = [f"{month:02}" for month in range(1, 12 + 1)]

In [34]:
def chunk_array(arr):
    """
    У Comtrade API есть ограничение, в запросе должно быть 
    не более 12 периодов. Эта функция делит все периоды на
    чанки по 12 частей
    """
    chunks = list()
    
    for i in range(0, len(arr), 12):
        chunks.append(list(arr[i:min(i + 12, len(arr))]))
    
    return chunks

# Заполняем значения

In [55]:
null_data_index = mirror_data[mirror_data.mirrorPrimaryValue.isna()].index

while len(null_data_index := mirror_data[mirror_data.mirrorPrimaryValue.isna()].index) > 0:
    print(f'Total null values: {len(null_data_index)}')
    
    reporter_code = mirror_data[mirror_data.mirrorPrimaryValue.isna()]['reporterCode'].iloc[0]
    periods_chunks = chunk_array(mirror_data[(mirror_data.reporterCode == reporter_code) & (mirror_data.mirrorPrimaryValue.isna())].period.unique())

    for periods in periods_chunks:
        send_request_time = time.time()
        
        params['period'] = ','.join(periods)
        params['partnerCode'] = reporter_code

        print(f'\t-> send request\t\t{reporter_code = }\t{periods[0]} - {periods[-1]}')

        try:
            response = requests.get(URL, params, timeout=60)    
        
        except Exception:
            continue

        get_response_time = time.time()

        if response.status_code == 403:
            params['subscription-key'] = input('Enter new subscription key: ')
            continue
        try:
            response_data = response.json()['data']
            df = pd.json_normalize(response_data)
    
        except Exception as exc:
            print(response.json())
            continue
    
        d_time = get_response_time - send_request_time
        print(f'\t<- response handled:\t{df.shape = }\td_time:{d_time:0.2f}')
        
        # Нужно ждать между запросами хотя бы 10 секунд.
        # Если мы получали ответ больше 10 секунд, то сразу отправляем новый
        time.sleep(max(0, 10 - d_time + 1))       
        
        for period in periods:
            condition = (mirror_data.period == period) & (mirror_data.reporterCode == reporter_code)
            
            try:
                filtered_df = df[df['period'] == period]
                
                reporters = list(filtered_df['reporterCode'])
                m_values = list(filtered_df['primaryValue'])
                m_wgts = list(filtered_df['netWgt'])

                mirror_data.loc[condition, 'mirrorPrimaryValue'] = str({country: value for country, value in zip(reporters, m_values)})
                mirror_data.loc[condition, 'mirrorNetWgt'] = str({country: weight for country, weight in zip(reporters, m_wgts)})
            
            except:
                mirror_data.loc[condition, 'mirrorPrimaryValue'] = str({})
                mirror_data.loc[condition, 'mirrorNetWgt'] = str({})

            null_data_index = mirror_data[mirror_data.mirrorPrimaryValue.isna()].index


Total null values: 7956
	-> send request		reporter_code = 70	201501 - 201512
	<- response handled:	df.shape = (517, 47)	d_time:5.90
	-> send request		reporter_code = 70	201601 - 201612
	<- response handled:	df.shape = (581, 47)	d_time:5.91
	-> send request		reporter_code = 70	201701 - 201712
	<- response handled:	df.shape = (773, 47)	d_time:5.72
	-> send request		reporter_code = 70	201801 - 201812
	<- response handled:	df.shape = (802, 47)	d_time:5.90
	-> send request		reporter_code = 70	201901 - 201912
	<- response handled:	df.shape = (868, 47)	d_time:7.04
	-> send request		reporter_code = 70	202001 - 202012
	<- response handled:	df.shape = (755, 47)	d_time:6.66
	-> send request		reporter_code = 70	202101 - 202112
	<- response handled:	df.shape = (858, 47)	d_time:7.57
	-> send request		reporter_code = 70	202201 - 202212
	<- response handled:	df.shape = (938, 47)	d_time:11.86
	-> send request		reporter_code = 70	202301 - 202312
	<- response handled:	df.shape = (891, 47)	d_time:10.10
To

Enter new subscription key:  9f1492872e0c48e4849fe94ab8cd1332


	-> send request		reporter_code = 52	201701 - 201712
	-> send request		reporter_code = 52	201801 - 201812
	<- response handled:	df.shape = (299, 47)	d_time:5.24
	-> send request		reporter_code = 52	201901 - 201912
	<- response handled:	df.shape = (355, 47)	d_time:9.91
	-> send request		reporter_code = 52	202001 - 202012
{'statusCode': 429, 'message': 'Rate limit is exceeded. Try again in 6 seconds.'}
	-> send request		reporter_code = 52	202101 - 202112
{'statusCode': 429, 'message': 'Rate limit is exceeded. Try again in 5 seconds.'}
	-> send request		reporter_code = 52	202201 - 202212
{'statusCode': 429, 'message': 'Rate limit is exceeded. Try again in 5 seconds.'}
	-> send request		reporter_code = 52	202301 - 202312
{'statusCode': 429, 'message': 'Rate limit is exceeded. Try again in 5 seconds.'}
Total null values: 4284
	-> send request		reporter_code = 762	201501 - 201512
{'statusCode': 429, 'message': 'Rate limit is exceeded. Try again in 5 seconds.'}
	-> send request		reporter_code

In [56]:
mirror_data[~mirror_data.mirrorPrimaryValue.isna()]

,period,reporterCode,mirrorPrimaryValue,mirrorNetWgt
0,201501,842,"{32: 20.0, 36: 14213.961, 40: 6345.066, 56: 47...","{32: 31.2, 36: 0.0, 40: 0.0, 56: 121257385.254..."
1,201502,842,"{352: 144687.791, 360: 2780.0, 372: 10427.245,...","{352: 203952.0, 360: 2813.176, 372: 5407.426, ..."
2,201503,842,"{604: 9238106.87, 579: 22806402.843, 191: 3798...","{604: 13515588.0, 579: 54169094.0, 191: 1002.0..."
3,201504,842,"{251: 32264324.748, 56: 51408687.716, 76: 753....","{251: 0.0, 56: 120008979.77, 76: 21.0, 724: 77..."
4,201505,842,"{76: 1060607.0, 276: 4850852.1, 152: 50.0, 40:...","{76: 2623903.223, 276: 1331880.0, 152: 1.42, 4..."
...,...,...,...,...
16195,202308,690,"{191: 4672.0, 480: 1520.005, 699: 37439.329, 7...","{191: 1045.0, 480: 228.0, 699: 12357.0, 710: 2..."
16196,202309,690,"{191: 2659.0, 480: 1636.697, 620: 11386.804, 6...","{191: 344.0, 480: 450.0, 620: 2119.0, 699: 45...."
16197,202310,690,"{156: 48000.0, 191: 4508.0, 450: 26889.777, 48...","{156: 50000.0, 191: 1121.11, 450: 18124.2, 480..."
16198,202311,690,"{450: 781.246, 480: 26442.854, 528: 1459.111, ...","{450: 484.27, 480: 18286.0, 528: 441.0, 699: 5..."


In [57]:
mirror_data

,period,reporterCode,mirrorPrimaryValue,mirrorNetWgt
0,201501,842,"{32: 20.0, 36: 14213.961, 40: 6345.066, 56: 47...","{32: 31.2, 36: 0.0, 40: 0.0, 56: 121257385.254..."
1,201502,842,"{352: 144687.791, 360: 2780.0, 372: 10427.245,...","{352: 203952.0, 360: 2813.176, 372: 5407.426, ..."
2,201503,842,"{604: 9238106.87, 579: 22806402.843, 191: 3798...","{604: 13515588.0, 579: 54169094.0, 191: 1002.0..."
3,201504,842,"{251: 32264324.748, 56: 51408687.716, 76: 753....","{251: 0.0, 56: 120008979.77, 76: 21.0, 724: 77..."
4,201505,842,"{76: 1060607.0, 276: 4850852.1, 152: 50.0, 40:...","{76: 2623903.223, 276: 1331880.0, 152: 1.42, 4..."
...,...,...,...,...
16195,202308,690,"{191: 4672.0, 480: 1520.005, 699: 37439.329, 7...","{191: 1045.0, 480: 228.0, 699: 12357.0, 710: 2..."
16196,202309,690,"{191: 2659.0, 480: 1636.697, 620: 11386.804, 6...","{191: 344.0, 480: 450.0, 620: 2119.0, 699: 45...."
16197,202310,690,"{156: 48000.0, 191: 4508.0, 450: 26889.777, 48...","{156: 50000.0, 191: 1121.11, 450: 18124.2, 480..."
16198,202311,690,"{450: 781.246, 480: 26442.854, 528: 1459.111, ...","{450: 484.27, 480: 18286.0, 528: 441.0, 699: 5..."


In [58]:
mirror_data.to_csv(f'graph data mirrored imp - {hs_code}.csv')

# Обьединяем статистику и зеркальную статистику

In [60]:
data.period = data.period.astype('str')
data.reporterCode = data.reporterCode.astype('int64')

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10950 entries, 0 to 0
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   period        10950 non-null  object
 1   reporterCode  10950 non-null  int64 
 2   primaryValue  10950 non-null  object
 3   netWgt        10950 non-null  object
dtypes: int64(1), object(3)
memory usage: 427.7+ KB


In [61]:
mirror_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16200 entries, 0 to 16199
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   period              16200 non-null  object
 1   reporterCode        16200 non-null  int64 
 2   mirrorPrimaryValue  16200 non-null  object
 3   mirrorNetWgt        16200 non-null  object
dtypes: int64(1), object(3)
memory usage: 632.8+ KB


In [83]:
result = mirror_data\
    .merge(data, on=['period', 'reporterCode'], how='left') \
    .fillna('{}') # Тогда точно получим данные зеркальной статистики

In [84]:
result

,period,reporterCode,mirrorPrimaryValue,mirrorNetWgt,primaryValue,netWgt
0,201501,842,"{32: 20.0, 36: 14213.961, 40: 6345.066, 56: 47...","{32: 31.2, 36: 0.0, 40: 0.0, 56: 121257385.254...","{124: 413074932.0, 484: 562456657.0, 826: 1478...","{124: 0.0, 484: 0.0, 826: 0.0, 156: 0.0, 392: ..."
1,201502,842,"{352: 144687.791, 360: 2780.0, 372: 10427.245,...","{352: 203952.0, 360: 2813.176, 372: 5407.426, ...","{124: 440461875.0, 410: 1943779.0, 344: 833824...","{124: 0.0, 410: 0.0, 344: 0.0, 784: 0.0, 376: ..."
2,201503,842,"{604: 9238106.87, 579: 22806402.843, 191: 3798...","{604: 13515588.0, 579: 54169094.0, 191: 1002.0...","{484: 496613044.0, 36: 4395521.0, 251: 8787662...","{484: 0.0, 36: 0.0, 251: 0.0, 152: 0.0, 591: 0..."
3,201504,842,"{251: 32264324.748, 56: 51408687.716, 76: 753....","{251: 0.0, 56: 120008979.77, 76: 21.0, 724: 77...","{156: 15464840.0, 344: 608236.0, 490: 2053497....","{156: 0.0, 344: 0.0, 490: 0.0, 188: 0.0, 604: ..."
4,201505,842,"{76: 1060607.0, 276: 4850852.1, 152: 50.0, 40:...","{76: 2623903.223, 276: 1331880.0, 152: 1.42, 4...","{410: 45811982.0, 76: 218399444.0, 214: 695832...","{410: 0.0, 76: 0.0, 214: 0.0, 764: 0.0, 724: 0..."
...,...,...,...,...,...,...
16195,202308,690,"{191: 4672.0, 480: 1520.005, 699: 37439.329, 7...","{191: 1045.0, 480: 228.0, 699: 12357.0, 710: 2...",{},{}
16196,202309,690,"{191: 2659.0, 480: 1636.697, 620: 11386.804, 6...","{191: 344.0, 480: 450.0, 620: 2119.0, 699: 45....",{},{}
16197,202310,690,"{156: 48000.0, 191: 4508.0, 450: 26889.777, 48...","{156: 50000.0, 191: 1121.11, 450: 18124.2, 480...",{},{}
16198,202311,690,"{450: 781.246, 480: 26442.854, 528: 1459.111, ...","{450: 484.27, 480: 18286.0, 528: 441.0, 699: 5...",{},{}


# Сохраняем результат

In [85]:
result.to_csv(f'export and mirrored export top 150 {hs_code} 2015-2023.csv')